In [ ]:
import pandas as pd
import numpy as np

# Importando as bases
df_vor = pd.read_excel('C:\\Users\\scosta\\Desktop\\Vorplan.xlsx') # Base de dados com a informção de planejamento para cada material
df_lx = pd.read_excel('C:\\Users\\scosta\\Desktop\\LX02 8510.XLSX') # Part numbers em estoque
df_sp = pd.read_excel('C:\\Users\\scosta\\Desktop\\SP99.xlsx') # Custo de cada part number 
df_06 = pd.read_excel('C:\\Users\\scosta\\Desktop\\vl06f.xlsx') # Part numbers aguardando separação logística

print("Tabelas importadas com sucesso")

# Validação dos dados carregados
assert 'A~Material' in df_vor.columns, "'A~Material' não encontrado na VORPLAN"
assert 'Material' in df_lx.columns, "'Material' não encontrado na LX02"
assert 'Material' in df_sp.columns, "'Material' não encontrado na SP99"
assert 'Material' in df_06.columns, "'Material' não encontrado na vl06f"

In [ ]:
# Função para normalizar os dados de material
def normalize_material(df, column_name):
    df[column_name] = df[column_name].astype(str).str.strip().str.upper()  # Converte para string, remove espaços e converte para maiúsculas
    df[column_name] = df[column_name].str.zfill(11)  # Preenche com zeros à esquerda para garantir 11 dígitos
    return df

# Normalizando a coluna específica de cada DataFrame
df_vor = normalize_material(df_vor, 'A~Material')
df_lx = normalize_material(df_lx, 'Material')
df_sp = normalize_material(df_sp, 'Material')
df_06 = normalize_material(df_06, 'Material')


# Converter colunas para o tipo correto
df_vor['B~Qtd.plan.'] = df_vor['B~Qtd.plan.'].astype(float)
df_vor['B~Rem/FimBas'] = pd.to_datetime(df_vor['B~Rem/FimBas'], format='%d/%m/%Y')
df_lx['Estoque disponível'] = df_lx['Estoque disponível'].astype(float)
#df_lx['Tipo de depósito'] = df_lx['Tipo de depósito'].astype(int)
df_06['Qtd.remessa'] = df_06['Qtd.remessa'].astype(float)


# Para o DataFrame df_sp
colunas_sp = ['Preço-padrão', 'Preço interno periódico', 'Unidade de preço']
for coluna in colunas_sp:
    df_sp[coluna] = df_sp[coluna].astype(float)

## Tratando VL06F ##

In [ ]:
# Agregação de valores
df_06 = df_06.groupby('Material')['Qtd.remessa'].sum()
df_06 = df_06.reset_index() # Resetar o índice para transformar o índice "Material" em uma coluna regular

## Tratando LX02 ##

## Manter 916 e verificar se está mesmo com valores = 0 ##


In [ ]:
# Verificando se de fato 'Tipo de depósito' 916 está com a quantidade zerada. Caso não esteja, talvez o df_06 não seja mais necessário
df_lx_916 = df_lx.drop(df_lx[df_lx['Tipo de depósito'] != 916].index)
qntde_916 = df_lx_916['Estoque disponível'].sum()

print(qntde_916)

In [ ]:
# Agregação de valores
df_lx = df_lx.groupby('Material')['Estoque disponível'].sum()
df_lx = df_lx.reset_index() # Resetar o índice para transformar o índice "Material" em uma coluna regular
df_lx['Estoque disponível'] = df_lx['Estoque disponível'].fillna(0)


df_lx = df_lx.merge(df_06[['Material', 'Qtd.remessa']], on='Material', how='outer')
df_lx['Qtd.remessa'] = df_lx['Qtd.remessa'].fillna(0)
df_lx['Total'] = df_lx['Estoque disponível'] + df_lx['Qtd.remessa'] 
df_lx.rename(columns={'Estoque disponível': 'Estoque LX02', 'Total': 'Estoque disponível'}, inplace=True)

#df_lx.to_excel('C:\\Users\\scosta\\Desktop\\LX02_pivot.xlsx')


## Tratando SP99 ##

In [ ]:
# Limpando informações desnecessárias
df_sp = df_sp.drop(df_sp[df_sp['Material'].isin(['nan'])].index) #remover linhas em que o material está vazio
#df_sp

# Inserindo coluna "Valor unitário" na SP99
df_sp['Valor unitário'] = np.where(df_sp['Preço interno periódico'] == 0, 
                                   df_sp['Preço-padrão'] / df_sp['Unidade de preço'], 
                                   df_sp['Preço interno periódico'] / df_sp['Unidade de preço'])

# Removendo duplicadas na SP99
df_sp.sort_values(by='Estoque total', ascending=False, inplace=True)
df_sp = df_sp.drop_duplicates(subset='Material', keep='first')

# Exporta o DataFrame para um arquivo Excel
df_sp.to_excel('C:\\Users\\scosta\\Desktop\\sp_filtrada.xlsx')

## Tratando  Vorplan ##

In [ ]:
# Trabalhando a vorplan
df_vor.rename(columns={'A~Material.1': 'Descrição'}, inplace=True)
df_vor = df_vor.merge(df_lx[['Material', 'Estoque disponível']], left_on='A~Material', right_on='Material', how='left')
df_vor['Estoque disponível'] = df_vor['Estoque disponível'].fillna(0)
df_vor = df_vor.merge(df_sp[['Material', 'Valor unitário']], left_on='A~Material', right_on='Material', how='left')
df_vor['Valor unitário'] = df_vor['Valor unitário'].fillna(0)
df_vor['Mês/Ano'] = df_vor['B~Rem/FimBas'].dt.strftime('%m/%Y')
df_vor.sort_values(by='B~Rem/FimBas', ascending=True, inplace=True)

## Calculando Valor em Trânsito ##

In [ ]:
# Passo 1: Calcular a média do estoque para cada código de material e adicionar ao DataFrame
df_vor['Estoque_médio'] = df_vor.groupby('A~Material')['Estoque disponível'].transform('mean')

# Alteração: Garantir que estamos utilizando o nome correto da coluna 'Estoque disponível'
df_vor['Estoque_médio'] = df_vor.groupby('A~Material')['Estoque disponível'].transform('mean')

# Passo 2: Calcular a soma cumulativa das quantidades planejadas para cada código
df_vor['Quantidade Acumulada'] = df_vor.groupby('A~Material')['B~Qtd.plan.'].cumsum()

# Passo 3: Calcular "Qtde em Trânsito" subtraindo o estoque médio da quantidade acumulada
df_vor['Qtde em Trânsito'] = df_vor['Quantidade Acumulada'] - df_vor['Estoque_médio']

# Ajustando a coluna 'Qtde em Trânsito' para garantir que os valores sejam cumulativos e corretos
df_vor['Qtde em Trânsito'] = df_vor.groupby('A~Material')['Qtde em Trânsito'].transform(lambda x: x.clip(lower=0).diff().fillna(x)) # Explicação 1

# Para garantir que os valores não sejam negativos e cumpram as condições iniciais:
df_vor['Qtde em Trânsito'] = df_vor['Qtde em Trânsito'].clip(lower=0) # Explicação 2

# Passo 4: Calcular o valor em trânsito
df_vor['Valor Total em Trânsito'] = df_vor['Qtde em Trânsito'] * df_vor['Valor unitário']

# Resultado final
df_vor.to_excel('C:\\Users\\scosta\\Desktop\\Trânsito_Vorplan.xlsx')

# Resultado final agrupado
resultado_agrupado = df_vor.groupby('Mês/Ano')['Valor Total em Trânsito'].sum().reset_index()
#resultado_agrupado.to_excel('C:\\Users\\scosta\\Desktop\\df_vor_2.xlsx')

### Explicação 1 ###

groupby('A~Material'): Agrupa o DataFrame pela coluna 'A~Material', o que permite aplicar uma função a cada grupo de valores que correspondem a um mesmo material. Isso é útil para operações que devem ser realizadas separadamente para cada tipo de material.

transform(lambda x: ...): A função transform é utilizada para aplicar uma operação a cada grupo sem alterar a estrutura original do df_vor. Ela mantém o índice original do DataFrame.

x.clip(lower=0): Dentro da função lambda, x.clip(lower=0) é usado para garantir que todos os valores em 'Qtde em Trânsito' sejam maiores que zero

diff(): Calcula a diferença entre cada elemento e o seu antecessor dentro do grupo. Isso é usado para determinar a variação na 'Qtde em Trânsito'.

fillna(x): Caso o .diff() gere algum valor NaN (como ocorrerá para o primeiro elemento de cada grupo, pois não há um elemento anterior para comparar), fillna(x) substitui esses NaNs pelo valor original de x antes das transformações. Isso significa que, para a primeira linha de cada grupo, o valor original de 'Qtde em Trânsito' é mantido.

### Explicação 2 ###

clip(lower=0): Esta função é usada novamente para garantir que, após todas as transformações, ajustes e cálculos, nenhum valor na coluna 'Qtde em Trânsito' seja negativo.